<a href="https://colab.research.google.com/github/TatianaNik/Vologda_championship/blob/main/Vologda_championship_final_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np 

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils, models

# Класс для конструирования модели нейронной сети
from tensorflow.keras.models import Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Input, concatenate

# Оптимизатор
from tensorflow.keras.optimizers import Adam

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Метрика
from sklearn.metrics import recall_score

from tensorflow.keras.callbacks import ModelCheckpoint



In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/train_dataset_Vologda.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/test_dataset_Vologda.csv")

In [ ]:
df_train.head(3)

,RecordNo,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
0,6145,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0,0,0,0,0
1,7006,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0,0,0,0,1
2,1124,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0,1,0,0,1


In [ ]:
df_train.shape

(4839, 17)

In [ ]:
df_train['Лайки на отзыв'].unique()

array([  0,   2,   1,  92,   7,   8,  22,   3,   5,  48,   4, 132, 111,
        15,  11,  21,  10,  12, 660,  73,  27,   6,  84,  50,  30,  42,
        20,  74,  19,  45,   9,  98,  59, 874,  18,  49,  16,  26,  31,
        68,  17,  14, 224, 114,  37, 251,  23,  56,  28, 156,  13, 168,
        29,  72,  38,  24,  65,  32,  57,  60,  35,  47,  41, 350,  69,
        33,  71, 107, 369,  63,  44,  43, 589, 105,  89,  58,  25, 153,
       332,  39, 249, 178,  54, 117, 200, 123,  34, 147,  75,  88,  78,
        40,  83,  87,  93, 791,  82,  90])

In [ ]:
df_test.shape

(2075, 12)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4839 entries, 0 to 4838
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   RecordNo                              4839 non-null   int64  
 1   Название книги                        4839 non-null   object 
 2   Автор                                 4839 non-null   object 
 3   Ссылка на литрес                      4839 non-null   object 
 4   Рейтинг                               4839 non-null   float64
 5   Количество оценок                     4839 non-null   int64  
 6   Количество отзывов                    4839 non-null   int64  
 7   Имя читателя                          4839 non-null   object 
 8   Оценка книги читателем (из 5 баллов)  4070 non-null   float64
 9   Отзыв                                 4839 non-null   object 
 10  Лайки на отзыв                        4839 non-null   int64  
 11  Дислайки на отзыв

In [ ]:
df_train["len_reviews"] = df_train["Отзыв"].map(len)

In [ ]:
df_test["len_reviews"] = df_test["Отзыв"].map(len)

In [ ]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
df_train.columns.values

array(['RecordNo', 'Название книги', 'Автор', 'Ссылка на литрес',
       'Рейтинг', 'Количество оценок', 'Количество отзывов',
       'Имя читателя', 'Оценка книги читателем (из 5 баллов)', 'Отзыв',
       'Лайки на отзыв', 'Дислайки на отзыв', 'Релевантность',
       'Таксономия релевантные', 'Таксономия не релевантные',
       'Длина отзыва', 'Ценности', 'len_reviews'], dtype=object)

In [ ]:
from sklearn import preprocessing  

label_encoder = preprocessing.LabelEncoder()  
# кодируем имена писателей в тренировочной и тестовой выборках метками
df_aftor_train = df_train['Автор']
df_aftor_test = df_test['Автор']
df = pd.concat([df_aftor_train, df_aftor_test])

df = label_encoder.fit_transform(df)

df.shape


(6914,)

In [ ]:
# нормализация данных 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.reshape(-1,1)) #  нормализация закодированных имен авторов

In [ ]:
df_train['Автор_label'] = df_scaled[:4839]  # в тренировочную выборку добавляем столбец с нормализованными именами авторов 

In [ ]:
df_test['Автор_label'] = df_scaled[-2075:]  # в тестовую выборку добавляем столбец с нормализованными именами авторов 
df_test['Автор_label'][:4]  # смотрим для примера первые 4 строки столбца   

0    0.383333
1    0.450000
2    0.450000
3    0.383333
Name: Автор_label, dtype: float64

In [ ]:
df_likes_train = df_train['Лайки на отзыв']
df_likes_test = df_test['Лайки на отзыв']
df = pd.concat([df_likes_train, df_likes_test])
print(df.shape)
scaler1 = MinMaxScaler()


(6914,)


In [ ]:
df_scaled = scaler1.fit_transform(np.array(df).reshape(-1,1)) #  нормализация  количества лайков
df_train['Лайки на отзыв'] = df_scaled[:4839] 
df_test['Лайки на отзыв'] = df_scaled[-2075:]

In [ ]:
df_dislikes_train = df_train['Дислайки на отзыв']
df_dislikes_test = df_test['Дислайки на отзыв']
df = pd.concat([df_dislikes_train, df_dislikes_test])

In [ ]:
df_scaled = scaler1.fit_transform(np.array(df).reshape(-1,1)) #  нормализация  количества дислайков
df_train['Дислайки на отзыв'] = df_scaled[:4839] 
df_test['Дислайки на отзыв'] = df_scaled[-2075:]

In [ ]:
# нормализуем значения рейтинга, количества отзывов, длину отзывов

In [ ]:
df_train['Рейтинг'] = scaler.fit_transform(np.array(df_train['Рейтинг']).reshape(-1,1))

In [ ]:
df_test['Рейтинг'] = scaler.fit_transform(np.array(df_test['Рейтинг']).reshape(-1,1))

In [ ]:
df_train['Количество отзывов'] = scaler.fit_transform(np.array(df_train['Количество отзывов']).reshape(-1,1))

In [ ]:
df_test['Количество отзывов'] = scaler.fit_transform(np.array(df_test['Количество отзывов']).reshape(-1,1))

In [ ]:
df_train['len_reviews'] = scaler.fit_transform(np.array(df_train['len_reviews']).reshape(-1,1))

In [ ]:
df_test['len_reviews'] = scaler.fit_transform(np.array(df_test['len_reviews']).reshape(-1,1))

In [ ]:
df_train['Оценка книги читателем (из 5 баллов)'].nunique()

6

In [ ]:
df_train.head(3)

,RecordNo,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,len_reviews,Автор_label
0,6145,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,0.833333,3922,0.193625,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0.0,0.006536,0,0,0,0,0,0.025884,0.383333
1,7006,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,0.777778,24719,1.000000,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0.0,0.002179,0,0,0,0,1,0.046789,0.383333
2,1124,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,0.666667,8032,0.333492,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0.0,0.000000,0,1,0,0,1,0.045960,0.383333


In [ ]:
# Настройка номеров столбцов df_train ( для удобства последующего использования)

AVTOR     = df_train.columns.get_loc('Автор_label')
RATING      = df_train.columns.get_loc('Рейтинг')
REVIEWS_NUMBER    = df_train.columns.get_loc('Количество отзывов')
MARK     = df_train.columns.get_loc('Оценка книги читателем (из 5 баллов)')
REVIEW      = df_train.columns.get_loc('Отзыв')
LIKES = df_train.columns.get_loc('Лайки на отзыв')
DISLIKES = df_train.columns.get_loc('Дислайки на отзыв') #  не использовались в данной версии
LEN_REVIEWS      = df_train.columns.get_loc('len_reviews')
RELEV = df_train.columns.get_loc('Релевантность')
TAX_RELEV = df_train.columns.get_loc('Таксономия релевантные')
TAX_NON_RELEV = df_train.columns.get_loc('Таксономия не релевантные')
LEN      = df_train.columns.get_loc('Длина отзыва')
VALUES      = df_train.columns.get_loc('Ценности')

In [ ]:
# Функция извлечения отзывов

def extract_text(row_list):
    result = []
    
    # Для всех строк таблицы
    for row in row_list:
        result.append(str(row[REVIEW]))
    
    # Возврат в виде массива
    return result

In [ ]:
# Извлечение текстов отзывов
text_train = extract_text(df_train.values) 
text_test = extract_text(df_test.values) 

In [ ]:
# предварительная обработка текста
from string import punctuation

def remove_punctuation(my_string):
  return "".join([ch if ch not in punctuation else " " for ch in my_string])

def remove_numbers(my_string):
  return "".join([i if not i.isdigit() else " " for i in my_string])

import re
def remove_multiple_spaces(my_string):
  return re.sub(r'\s+', ' ', my_string, flags=re.I)

preprocessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))))

text_processed = ['']

df_train['text_processed'] = list(map(preprocessing, text_train))
df_test['text_processed'] = list(map(preprocessing, text_test))
df_train.head(3)


,RecordNo,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,len_reviews,Автор_label,text_processed
0,6145,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,0.833333,3922,0.193625,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0.0,0.006536,0,0,0,0,0,0.025884,0.383333,рекомендую книгу в прочтению прослушиванию ест...
1,7006,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,0.777778,24719,1.000000,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0.0,0.002179,0,0,0,0,1,0.046789,0.383333,удивительно что сейчас возникает такая литерат...
2,1124,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,0.666667,8032,0.333492,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0.0,0.000000,0,1,0,0,1,0.045960,0.383333,душевно жизненно чувственно проникновенно исто...


In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dict
!pip install DAWG-Python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 8.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=58b9b55a39d47b236cedc55bb5828dfd6b474c7a779fbc07f3de0819b341ce11
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pymorphy2-dict (from versions: none)
ERROR: No matching distribution found for pymorphy2-dict
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from tqdm.auto import tqdm, trange
import pymorphy2
nltk.download('stopwords')
nltk.download('punkt')
morph = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

# лемматизация тренировочной выборки
lemm_texts_column = [ ]
for text in tqdm(df_train['text_processed']):
  tokens = word_tokenize(text)
  lemmed_tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in russian_stopwords]
  text = " ".join(lemmed_tokens)
  lemm_texts_column.append(text)
#  создаем новую колонку
df_train["text_lemmed"] = lemm_texts_column

# лемматизация тестовой выборки
lemm_texts_column = [ ]
for text in tqdm(df_test['text_processed']):
  tokens = word_tokenize(text)
  lemmed_tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in russian_stopwords]
  text = " ".join(lemmed_tokens)
  lemm_texts_column.append(text)
#  создаем новую колонку
df_test["text_lemmed"] = lemm_texts_column

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  0%|          | 0/4839 [00:00<?, ?it/s]

  0%|          | 0/2075 [00:00<?, ?it/s]

In [ ]:
# Извлечение тренировочных записей 
text_data = df_train['text_lemmed'].tolist()
# Извлечение тестовых записей 
test_data = df_test['text_lemmed'].tolist()

In [ ]:
# Используем встроенный в Keras токенизатор для разбиения текста отзыва и построения частотного словаря
tokenizer = Tokenizer(num_words=3000, # объем словаря
                      filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # убираемые из текста ненужные символы 
                      lower=True, # приведение слов к нижнему регистру
                      split=' ', # разделитель слов
                      oov_token='unknown', # токен для слов, которые не вошли в словарь
                      char_level=False     # указание разделять по словам, а не по единичным символам
                      )

# Построение частотного словаря по отзывам
tokenizer.fit_on_texts(text_data+test_data)

In [ ]:
items = list(tokenizer.word_index.items())       # Получение индексов слов
print(items[:50])                                # Посмотр 50 самых часто встречающихся слов
print("Размер словаря", len(items))              # Длина словаря

[('unknown', 1), ('книга', 2), ('очень', 3), ('автор', 4), ('это', 5), ('«', 6), ('»', 7), ('читать', 8), ('жизнь', 9), ('человек', 10), ('герой', 11), ('который', 12), ('история', 13), ('один', 14), ('время', 15), ('такой', 16), ('прочитать', 17), ('роман', 18), ('свой', 19), ('сюжет', 20), ('спасибо', 21), ('понравиться', 22), ('произведение', 23), ('язык', 24), ('хороший', 25), ('интересный', 26), ('весь', 27), ('написать', 28), ('первый', 29), ('прочтение', 30), ('хотеться', 31), ('рекомендовать', 32), ('просто', 33), ('дыхание', 34), ('конец', 35), ('наш', 36), ('каждый', 37), ('этот', 38), ('любовь', 39), ('судьба', 40), ('сам', 41), ('тот', 42), ('читаться', 43), ('год', 44), ('событие', 45), ('описание', 46), ('мой', 47), ('прекрасный', 48), ('мочь', 49), ('душа', 50)]
Размер словаря 19367


In [ ]:
# Преобразование текстов в последовательность индексов согласно частотному словарю
text_seq = tokenizer.texts_to_sequences(text_data)

test_seq = tokenizer.texts_to_sequences(test_data)

In [ ]:
# Преобразование последовательностей индексов в bag of words
x_train_text = tokenizer.sequences_to_matrix(text_seq)
x_train_text.shape
x_test_text = tokenizer.sequences_to_matrix(test_seq)
x_test_text.shape

(2075, 3000)

In [ ]:
# Проверка результата
print(x_train_text.shape)
print(x_train_text[0][0:100])

(4839, 3000)
[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [ ]:
def extract_row_data(row):
  
    # Извлечение и преобразование данных
    avtor = np.array([row[AVTOR]])                          # автор
    rating = np.array([row[RATING]])                         # рейтинг
    rev_number = np.array([row[REVIEWS_NUMBER]])             # количество отзывов
    like = np.array([row[LIKES]])
    
    mark = utils.to_categorical(row[MARK], 6)               # оценка книги читателем в формате ohe
    len_rev = np.array([row[LEN_REVIEWS]])                   # длина отзыва
                                         
    # Объединение всех входных данных в один общий вектор
    x_data = np.hstack([avtor,
                        rating, 
                        rev_number,
                        like,
                        mark,
                        len_rev])
    
     
    y_data = np.hstack([row[RELEV],
                        row[TAX_RELEV],
                        row[TAX_NON_RELEV],
                        row[LEN],
                        row[VALUES]])
    
    # Возврат входных данных и выходных
    return x_data, y_data


# Создание общей выборки
def construct_train_data(row_list):
    x_data = []
    y_data = []
    
    for row in row_list:
        x, y = extract_row_data(row)
        x_data.append(x)
        y_data.append(y)

    return np.array(x_data), np.array(y_data)

In [ ]:
# Формирование выборки из загруженного набора данных    
x_train, y_train = construct_train_data(df_train.values)

In [ ]:
# строим сеть
input1 = Input((x_train.shape[1],))

input2 = Input((x_train_text.shape[1],))

x1 = input1                              # Ветка 1  
x1 = Dense(5000, activation="relu")(x1)
x1 = Dense(3000, activation="relu")(x1)

x1 = Dense(2000, activation="relu")(x1)
x1 = Dense(1000, activation="relu")(x1)

x1 = Dense(500, activation="relu")(x1)


x2 = input2                               # Ветка 2  

x2 = Dense(4000, activation="relu")(x2)
x2 = Dense(3000, activation="relu")(x2)

x2 = Dense(2000, activation="relu")(x2)
x2 = Dense(1000, activation="relu")(x2)
x2 = Dense(500, activation="relu")(x2)


x = concatenate([x1, x2])             # Объединение веток
#x = BatchNormalization()(x) 

x = Dense(500, activation='relu')(x)       # Промежуточный слой
x = Dense(1000, activation='relu')(x)       # Промежуточный слой
x = Dense(500, activation='relu')(x)       # Промежуточный слой

x = Dense(100, activation='relu')(x)       # Промежуточный слой

x = BatchNormalization()(x) 
x = Dense(5, activation='sigmoid')(x)       

# В Model передаются входы и выход
model_final = Model((input1, input2), x)


In [ ]:
model_save_path = '/content/model_9.h5' 
checkpoint_callback = ModelCheckpoint(model_save_path, monitor='val_recall', save_best_only=True, mode='max', verbose=1)

In [ ]:
model_final.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['Recall'])
# Вывод сводки
model_final.summary()

# Обучение модели с заданными параметрами
  
history = model_final.fit([x_train, x_train_text], 
                           y_train,
                           batch_size=128, 
                           epochs=30, 
                           validation_split=0.2,
                           shuffle=True,
                           callbacks=[checkpoint_callback],
                           verbose=1)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 11)]         0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 3000)]       0           []                               
                                                                                                  
 dense_57 (Dense)               (None, 5000)         60000       ['input_9[0][0]']                
                                                                                                  
 dense_62 (Dense)               (None, 4000)         12004000    ['input_10[0][0]']               
                                                                                            

In [ ]:
# Настройка номеров столбцов df_test

AVTOR     = df_test.columns.get_loc('Автор_label')
RATING      = df_test.columns.get_loc('Рейтинг')
REVIEWS_NUMBER    = df_test.columns.get_loc('Количество отзывов')
MARK     = df_test.columns.get_loc('Оценка книги читателем (из 5 баллов)')
REVIEW      = df_test.columns.get_loc('Отзыв')
LIKES = df_test.columns.get_loc('Лайки на отзыв')
DISLIKES = df_test.columns.get_loc('Дислайки на отзыв')
LEN_REVIEWS      = df_test.columns.get_loc('len_reviews')

In [ ]:
model = models.load_model('/content/model_9.h5')  # загружаем сохраненную модель 

In [ ]:
def extract_row_test(row):
  
    # Извлечение и преобразование данных
    avtor = np.array([row[AVTOR]])                          # автор
    rating = np.array([row[RATING]])                         # рейтинг
    rev_number = np.array([row[REVIEWS_NUMBER]])             # количество отзывов
    like = np.array([row[LIKES]])
    mark = utils.to_categorical(row[MARK], 6)               # оценка книги читателем в формате ohe
    len_rev = np.array([row[LEN_REVIEWS]])                   # длина отзыва
                                         
    # Объединение всех входных данных в один общий вектор
    x_data = np.hstack([avtor,
                        rating, 
                        rev_number,
                        like,
                        mark,
                        len_rev])
       
     
    # Возврат входных данных
    return x_data


# Создание общей выборки
def construct_test_data(row_list):
    x_data = []
       
    for row in row_list:
        x = extract_row_test(row)
        x_data.append(x)
        
    return np.array(x_data)

In [ ]:
x_test = construct_test_data(df_test.values)

In [ ]:
model.save('/content/drive/MyDrive/model_9.h5')

In [ ]:
y_pred = model.predict([x_test, x_test_text])  #  предсказание на тестовом наборе
print(y_pred.shape)

(2075, 5)


In [ ]:
print(y_pred[:5])

[[0.89000964 0.31399053 0.02563559 0.98022586 0.9801793 ]
 [0.53913814 0.9925961  0.04814338 0.1787027  0.04507709]
 [0.12664598 0.0299376  0.04813948 0.08168605 0.9939599 ]
 [0.05664046 0.02222645 0.13758694 0.05234849 0.21108161]
 [0.8801666  0.95278126 0.04126056 0.17527576 0.01940012]]


In [ ]:
y = np.around(y_pred[:])

In [ ]:
y = np.array(y, dtype=np.int64)
y[10]

array([0, 1, 0, 1, 1])

In [ ]:
column_values = ['Релевантность', 'Таксономия релевантные', 'Таксономия не релевантные', 'Длина отзыва', 'Ценности']
df = pd.DataFrame(data = y, 
                  columns = column_values)
df.index = df_test['RecordNo']

df.head()

,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
RecordNo,,,,,
3366,1,0,0,1,1
3952,1,1,0,0,0
6852,0,0,0,0,1
4586,0,0,0,0,0
4677,1,1,0,0,0


In [ ]:
df.to_csv("Vologda_prediction9.csv")